In [60]:
import random
import string

# Define the alphabet, I specifically add these chars because the length (modulus) will be prime, thus we always have an inverse matrix
ALPHABET = string.ascii_letters + string.digits + "{}_?!"
n = len(ALPHABET)

# Define the message
message = "Applied_Cryptography{Knowing_the_plaintext_and_ciphertext_is_easy_if_you_know_how_to_solve_system_of_equations_mod_n}"
# Function to convert string to matrix in GF(n)
def matrix_to_string(m):
    l = m.ncols()
    s = ""
    for i in range(l*l):
        s += ALPHABET[int(m[i // l, i % l])]
    return s

# Function to create the message matrix and its corresponding key matrix
def create_pubkey(message: str):
    n_m = len(message)
    n_col = 0
    while True:
        if n_col^2 > n_m:
            message += "!" * (n_col^2 - n_m)
            break
        else:
            if n_col^2 == n_m:
                break
            else:
                n_col += 1
    
    while True:
        key = random_matrix(Zmod(n), n_col, n_col)
        if key.determinant() != 0 and gcd(key.determinant(), n) == 1:
            break
    M = []
    for i in range(n_col):
        m_i = []
        for j in range(n_col):
            m_i.append(ALPHABET.index(message[i*n_col + j]))
        M.append(m_i)
    M = Matrix(Zmod(n), M)
    return M, key

# Encryption (fairly easy to write)
def encrypt(m: str, k):
    return m*k

# Decryption
def decrypt(c, k):
    return c*k^-1

# Function to test 
def test_hill_cipher(rounds):
    score = 0
    for i in range(rounds):
        len = random.randint(0, 1000)
        m = ""
        for _ in range(len):
            m += random.choice(ALPHABET)

        m_matrix, key = create_pubkey(m)

        ciphertext = encrypt(m_matrix, key)
        plaintext = matrix_to_string(decrypt(ciphertext, key))

        if (m in plaintext):
            score += 1
            
    print("Final result : " + str(score) + "/" + str(rounds))
    print("Accuracy : " + str(score*100/rounds.round()) + "%")


#test_hill_cipher(10)
m_matrix, key = create_pubkey(message)
ciphertext = encrypt(m_matrix, key)
plaintext = decrypt(ciphertext, key)

assert m_matrix == plaintext

# Return the key
def known_plaintext_attack(plaintext, ciphertext):
    key = []
    size = plaintext.dimensions()[0]
    
    # Form an extended matrix for each columns
    for i in range(size):
        M = []
        for j in range(size):
            temp = list(plaintext[j])
            temp.append(ciphertext[j][i])
            M.append(temp)
        M = Matrix(GF(n), M)
        key_col = M.echelon_form()
        key.append(list(key_col.column(size)))
    return Matrix(GF(n), key).transpose()
        

key_attack = known_plaintext_attack(plaintext, ciphertext)


print(matrix_to_string(ciphertext * (key_attack^-1)))





Applied_Cryptography{Knowing_the_plaintext_and_ciphertext_is_easy_if_you_know_how_to_solve_system_of_equations_mod_n}!!!!
